# SMS Spam Detection Modeling

In [1]:
import string
import re
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/smlovullo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/smlovullo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/smlovullo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading the Data

Import raw data from file

In [3]:
with open('../data/SMSSpamCollection.txt', 'r') as file_stream:
    sms_messages_raw = file_stream.readlines()

Collect labels and messages from imported data

In [4]:
labels = [re.search("(^.*)\t", message).group(1) for message in sms_messages_raw]
sms_messages = [message[message.index('\t')+1:-1] for message in sms_messages_raw]

Stitch the data together in a data frame

In [5]:
messages_df = pd.DataFrame(data={'Labels': labels, 'Messages': sms_messages})
messages_df['Labels'] = messages_df['Labels'].apply(lambda x: 1 if x == 'spam' else 0)
messages_df

,Labels,Messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,1,This is the 2nd time we have tried 2 contact u...
5570,0,Will ü b going to esplanade fr home?
5571,0,"Pity, * was in mood for that. So...any other s..."
5572,0,The guy did some bitching but I acted like i'd...


In [6]:
def get_punc_count(s: str) -> int:
    count = 0
    for char in s:
        count += 1 if char in string.punctuation else 0
    return count

def get_numeric_count(s: str) -> int:
    count = 0
    tokens = word_tokenize(s)
    for word in tokens:
        count += 1 if word.isnumeric() else 0
    return count

def get_uppercase_count(s: str) -> int:
    count = 0
    for char in s:
        count += 1 if char.isupper() else 0
    return count

Construct dataset of various countable features from SMS Messages

In [7]:
counts_dict = {
    'character_counts': [],
    'punc_counts': [],
    'numeric_counts': [],
    'uppercase_counts': []
}
for message in messages_df['Messages']:
    counts_dict['character_counts'].append(len(message))
    counts_dict['punc_counts'].append(get_punc_count(message))
    counts_dict['numeric_counts'].append(get_numeric_count(message))
    counts_dict['uppercase_counts'].append(get_uppercase_count(message))

In [8]:
counts_df = messages_df[['Labels']].join(pd.DataFrame(counts_dict))
counts_df

,Labels,character_counts,punc_counts,numeric_counts,uppercase_counts
0,0,111,9,0,3
1,0,29,6,0,2
2,1,155,6,3,10
3,0,49,6,0,2
4,0,61,2,0,2
...,...,...,...,...,...
5569,1,160,8,3,9
5570,0,36,1,0,1
5571,0,57,7,0,2
5572,0,125,1,0,2


## Model Set 1: Building Models Using String Feature Counts

In [9]:
X = counts_df.drop(labels=['Labels'], axis=1)
y = counts_df['Labels']

In [10]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=256)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

In [11]:
model_metrics = {'k_folds': n_splits, 'metrics': {}}

In [12]:
clf_lr = Pipeline([
    ('normalize', Normalizer()),
    ('scale', RobustScaler()),
    ('classify', LogisticRegression()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    # remove outliers for logistic regression training
    indices_with_outliars = set()
    for index in X_train[X_train['character_counts'] > X_train['character_counts'].quantile(0.99)].index.to_list():
        indices_with_outliars.add(index)
    for index in X_train[X_train['punc_counts'] > X_train['punc_counts'].quantile(0.99)].index.to_list():
        indices_with_outliars.add(index)
    for index in X_train[X_train['numeric_counts'] > X_train['numeric_counts'].quantile(0.99)].index.to_list():
        indices_with_outliars.add(index)
    for index in X_train[X_train['uppercase_counts'] > X_train['uppercase_counts'].quantile(0.99)].index.to_list():
        indices_with_outliars.add(index)
    indices_with_outliars = list(indices_with_outliars)

    clf_lr.fit(X_train.drop(labels=indices_with_outliars, axis=0), y_train.drop(labels=indices_with_outliars, axis=0))
    y_hat = clf_lr.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Logistic Regression Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")


Logistic Regression Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9033405806948409 and its std: 0.005573357835380953
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.549485757930491 and its std: 0.03778797998883952


In [13]:
clf_dt = Pipeline([
    ('scale', RobustScaler()),
    ('classify', DecisionTreeClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_dt.fit(X_train, y_train)
    y_hat = clf_dt.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Decision Tree Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Decision Tree Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9441580730071013 and its std: 0.0066269627495905105
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.7892391638510784 and its std: 0.02646930932293095


In [14]:
clf_nb = Pipeline([
    ('classify', MultinomialNB())
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_nb.fit(X_train, y_train)
    y_hat = clf_nb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Multinomial Naive Bayes Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Multinomial Naive Bayes Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.909845339292275 and its std: 0.004168673996548207
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.6484110463043381 and its std: 0.022972931667944056


In [15]:
k_neighbor_min = 1
k_neighbor_max = 20

knn_classifier_results = {}
max_accuracy = 0
max_f1_score = 0 
for k_neighbors in range(k_neighbor_min,k_neighbor_max+1):
    clf_knn = Pipeline([
        ('scale', RobustScaler()),
        ('classify', KNeighborsClassifier(n_neighbors=k_neighbors)),
    ])
    cv_accuracies = np.array([])
    cv_spam_f1_scores = np.array([])
    for train_index, val_index in skf.split(X_train_full, y_train_full):
        X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
        y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

        clf_knn.fit(X_train, y_train)
        y_hat = clf_nb.predict(X_val)
        report_results = classification_report(y_val, y_hat, output_dict=True)
        cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
        cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

    mean_accuracy = cv_accuracies.mean()
    std_accuracy = cv_accuracies.std()
    mean_f1_score = cv_spam_f1_scores.mean()
    std_f1_score = cv_spam_f1_scores.std()

    if max_accuracy <= mean_accuracy:
        max_accuracy = mean_accuracy
    if max_f1_score <= mean_f1_score:
        max_f1_score = mean_f1_score

    knn_classifier_results[k_neighbors] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

for k_neighbor_val, cv_results in knn_classifier_results.items():
    if cv_results['cv_f1_score'] >= max_f1_score:
        model_name = f"K-Nearest Neighbors Classifier (k={k_neighbor_val})"
        model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

        print(model_name)
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {cv_results['cv_accuracy']} and its std: {cv_results['cv_accuracy_std']}")
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {cv_results['cv_f1_score']} and its std: {cv_results['cv_f1_score_std']}")


K-Nearest Neighbors Classifier (k=15)
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9105220113441339 and its std: 0.010546870688534165
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.6533606765567294 and its std: 0.03517929137317372


In [16]:
clf_svc = Pipeline([
    ('scale', RobustScaler()),
    ('classify', SVC()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_svc.fit(X_train, y_train)
    y_hat = clf_svc.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Support Vector Machine Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Support Vector Machine Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9564949444620595 and its std: 0.006540073041995297
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8185636050415142 and its std: 0.02968020532000147


In [17]:
clf_mlp = Pipeline([
    ('scale', RobustScaler()),
    ('classify', MLPClassifier(max_iter=1000)),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_mlp.fit(X_train, y_train)
    y_hat = clf_mlp.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Multilayer Perceptron Neural Network Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Multilayer Perceptron Neural Network Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9580619347435491 and its std: 0.005242553316435197
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8314374510339206 and its std: 0.020066421662798938


In [18]:
clf_rf = Pipeline([
    ('scale', RobustScaler()),
    ('classify', RandomForestClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_rf.fit(X_train, y_train)
    y_hat = clf_rf.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Random Forest Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Random Forest Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9569428716663395 and its std: 0.004721475912709519
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8326990322300594 and its std: 0.018401993010771493


In [19]:
clf_et = Pipeline([
    ('scale', RobustScaler()),
    ('classify', ExtraTreesClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_et.fit(X_train, y_train)
    y_hat = clf_et.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Extra Trees Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Extra Trees Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.955372358361895 and its std: 0.0035601586411803674
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8288901181033298 and its std: 0.012553233296701334


In [20]:
clf_gb = Pipeline([
    ('scale', RobustScaler()),
    ('classify', GradientBoostingClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_gb.fit(X_train, y_train)
    y_hat = clf_gb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Gradient Boosting Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Gradient Boosting Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9562664512589774 and its std: 0.00699617888979639
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8249177997671799 and its std: 0.03176069546929763


In [21]:
clf_xgb = Pipeline([
    ('scale', RobustScaler()),
    ('classify', XGBClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_xgb.fit(X_train, y_train)
    y_hat = clf_xgb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "XGBoost Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

XGBoost Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9585106168813194 and its std: 0.00646126629165489
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8362791895717742 and its std: 0.024358836669605134


In [22]:
decision_scores = ['cv_accuracy', 'cv_f1_score']
for score_type in decision_scores:
    models_by_score = {}
    for model_name, metrics in model_metrics['metrics'].items():
        models_by_score[metrics[score_type]] = model_name
    scores = np.array(list(models_by_score.keys()))
    max_score = scores.max()
    best_scoring_model_name = models_by_score[max_score]
    print(f"Model with best {score_type}: {best_scoring_model_name}")
    print(f"\tAverage Accuracy: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy_std']}")
    print(f"\tAverage F1-Score: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score_std']}")


Model with best cv_accuracy: XGBoost Classifier
	Average Accuracy: 0.9585106168813194 with Standard Deviation: 0.00646126629165489
	Average F1-Score: 0.8362791895717742 with Standard Deviation: 0.024358836669605134
Model with best cv_f1_score: XGBoost Classifier
	Average Accuracy: 0.9585106168813194 with Standard Deviation: 0.00646126629165489
	Average F1-Score: 0.8362791895717742 with Standard Deviation: 0.024358836669605134


Before hyperparameter tuning and by only using a dataset that contains various counted metrics about each sms message, it is possible to train a model with a high estimated accuracy and f1-score. Multiple runs of examining model cross-validation results found that the following models often performed better than the rest:

* Random Forest Classifier
* XGBoost Classifier
* Gradient Boosting Classifier
* Extra Trees Classifier

The random forest classifier more often outperformed the other models, so it was the chosen algorithm to construct the model

In [23]:
classifier = Pipeline([
    ('scale', RobustScaler()),
    ('classify', RandomForestClassifier()),
])
param_grid = {
    'classify__n_estimators': [50, 100, 250, 400, 500],
    'classify__max_features': ['sqrt', 'log2'],
    'classify__min_samples_split': [2, 5, 10],
    'classify__min_samples_leaf': [1, 2, 4],
    'classify__bootstrap': [True, False]
}

grid = GridSearchCV(classifier, param_grid, cv=n_splits, scoring='f1')
grid.fit(X_train_full, y_train_full)

print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)

Best parameters: {'classify__bootstrap': True, 'classify__max_features': 'sqrt', 'classify__min_samples_leaf': 1, 'classify__min_samples_split': 2, 'classify__n_estimators': 500}
Best cross-validation score: 0.8503556806032002


In [24]:
classifier = Pipeline([
    ('scale', RobustScaler()),
    ('classify', RandomForestClassifier(
        n_estimators=grid.best_params_['classify__n_estimators'],
        max_features=grid.best_params_['classify__max_features'],
        min_samples_split=grid.best_params_['classify__min_samples_split'],
        min_samples_leaf=grid.best_params_['classify__min_samples_leaf'],
        bootstrap=grid.best_params_['classify__bootstrap']
    ))
])
classifier.fit(X_train_full, y_train_full)
y_hat = classifier.predict(X_test)
report_results = classification_report(y_test, y_hat, output_dict=True)
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))
print(f"Accuracy: {report_results['accuracy']}")
print(f"F1-Score: {report_results['1']['f1-score']}")

[[953  13]
 [ 24 125]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       966
           1       0.91      0.84      0.87       149

    accuracy                           0.97      1115
   macro avg       0.94      0.91      0.93      1115
weighted avg       0.97      0.97      0.97      1115

Accuracy: 0.9668161434977578
F1-Score: 0.8710801393728224


In [25]:
classifier = Pipeline([
    ('scale', RobustScaler()),
    ('classify', GradientBoostingClassifier()),
])
param_grid = {
    'classify__n_estimators': [50, 100, 150],
    'classify__learning_rate': [0.01, 0.1, 1],
    'classify__max_depth': [1, 3, 5, 7]
}

grid = GridSearchCV(classifier, param_grid, cv=n_splits, scoring='f1')
grid.fit(X_train_full, y_train_full)

print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)

Best parameters: {'classify__learning_rate': 0.1, 'classify__max_depth': 3, 'classify__n_estimators': 150}
Best cross-validation score: 0.8396048177490301


In [26]:
classifier = Pipeline([
    ('scale', RobustScaler()),
    ('classify', GradientBoostingClassifier(
        learning_rate=grid.best_params_['classify__learning_rate'],
        n_estimators=grid.best_params_['classify__n_estimators'],
        max_depth=grid.best_params_['classify__max_depth']
    )),
])
classifier.fit(X_train_full, y_train_full)
y_hat = classifier.predict(X_test)
report_results = classification_report(y_test, y_hat, output_dict=True)
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))
print(f"Accuracy: {report_results['accuracy']}")
print(f"F1-Score: {report_results['1']['f1-score']}")

[[947  19]
 [ 25 124]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       966
           1       0.87      0.83      0.85       149

    accuracy                           0.96      1115
   macro avg       0.92      0.91      0.91      1115
weighted avg       0.96      0.96      0.96      1115

Accuracy: 0.9605381165919282
F1-Score: 0.8493150684931506


## Model Set 2: Building Models Based on Word Content Using Count Vectorizers

In [27]:
class TextCleaner():

    def __init__(self):
        self.punct_table = str.maketrans('', '', string.punctuation)
        self.stopwords = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.transformations = [
            self._strip_punct,
            self._convert_to_lowercase,
            self._remove_stopwords,
            self._remove_numbers,
            self._remove_special_characters,
            self._lemmatize
        ]

    def fit(self, X, y):
        return self

    def fit_transform(self, X, y):
        return self.clean_text(X)

    def transform(self, X):
        return self.clean_text(X)

    def clean_text(self, X):
        cleaned_text = []
        for text in X:
            for transformation in self.transformations:
                text = transformation(text)
            cleaned_text.append(text)
        return cleaned_text

    def _strip_punct(self, text: str) -> str:
        return text.translate(self.punct_table)

    def _convert_to_lowercase(self, text: str) -> str:
        return text.lower()

    def _remove_stopwords(self, text: str) -> str:
        words = nltk.word_tokenize(text)
        words = [w for w in words if w not in self.stopwords]
        return ' '.join(words)

    def _remove_numbers(self, text: str) -> str:
        words = nltk.word_tokenize(text)
        words = [w for w in words if not re.search(r'\d', w)]
        return ' '.join(words)

    def _remove_special_characters(self, text: str) -> str:
        pattern = r'[^a-zA-Z0-9\s]'
        return re.sub(pattern, '', text)

    def _lemmatize(self, text: str) -> str:
        words = nltk.word_tokenize(text)
        words = [self.lemmatizer.lemmatize(w) for w in words]
        return ' '.join(words)

In [28]:
text_df = messages_df[['Labels','Messages']]
text_df

,Labels,Messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,1,This is the 2nd time we have tried 2 contact u...
5570,0,Will ü b going to esplanade fr home?
5571,0,"Pity, * was in mood for that. So...any other s..."
5572,0,The guy did some bitching but I acted like i'd...


In [29]:
X = text_df['Messages']
y = text_df['Labels']

In [30]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=256)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

In [31]:
model_metrics = {'k_folds': n_splits, 'metrics': {}}

In [32]:
clf_lr = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('normalize', Normalizer()),
    ('classify', LogisticRegression()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_lr.fit(X_train, y_train)
    y_hat = clf_lr.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Logistic Regression Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")


Logistic Regression Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9558197822771813 and its std: 0.0036556890170207144
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8052251994736557 and its std: 0.019279604032540205


In [33]:
clf_dt = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', DecisionTreeClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_dt.fit(X_train, y_train)
    y_hat = clf_dt.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Decision Tree Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Decision Tree Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.954475245730851 and its std: 0.009497171207188744
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.826114364268243 and its std: 0.039747133668020546


In [34]:
clf_nb = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', MultinomialNB())
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_nb.fit(X_train, y_train)
    y_hat = clf_nb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Multinomial Naive Bayes Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Multinomial Naive Bayes Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9614271765990751 and its std: 0.003849860674290464
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8317583829316982 and its std: 0.019241701518066836


In [35]:
k_neighbor_min = 1
k_neighbor_max = 20

knn_classifier_results = {}
max_accuracy = 0
max_f1_score = 0 
for k_neighbors in range(k_neighbor_min,k_neighbor_max+1):
    clf_knn = Pipeline([
        ('text_cleaner', TextCleaner()),
        ('vectorizer', TfidfVectorizer()),
        ('scale', RobustScaler(with_centering=False)),
        ('classify', KNeighborsClassifier(n_neighbors=k_neighbors)),
    ])
    cv_accuracies = np.array([])
    cv_spam_f1_scores = np.array([])
    for train_index, val_index in skf.split(X_train_full, y_train_full):
        X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
        y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

        clf_knn.fit(X_train, y_train)
        y_hat = clf_nb.predict(X_val)
        report_results = classification_report(y_val, y_hat, output_dict=True)
        cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
        cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

    mean_accuracy = cv_accuracies.mean()
    std_accuracy = cv_accuracies.std()
    mean_f1_score = cv_spam_f1_scores.mean()
    std_f1_score = cv_spam_f1_scores.std()

    if max_accuracy <= mean_accuracy:
        max_accuracy = mean_accuracy
    if max_f1_score <= mean_f1_score:
        max_f1_score = mean_f1_score

    knn_classifier_results[k_neighbors] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

for k_neighbor_val, cv_results in knn_classifier_results.items():
    if cv_results['cv_f1_score'] >= max_f1_score:
        model_name = f"K-Nearest Neighbors Classifier (k={k_neighbor_val})"
        model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

        print(model_name)
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {cv_results['cv_accuracy']} and its std: {cv_results['cv_accuracy_std']}")
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {cv_results['cv_f1_score']} and its std: {cv_results['cv_f1_score_std']}")


K-Nearest Neighbors Classifier (k=4)
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9715186242092072 and its std: 0.002074758042888417
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8810973434797049 and its std: 0.0098913648133967


In [36]:
clf_svc = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', SVC()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_svc.fit(X_train, y_train)
    y_hat = clf_svc.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Support Vector Machine Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Support Vector Machine Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9744329191264918 and its std: 0.005995136631526237
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.895200928865399 and its std: 0.027362142586644163


In [37]:
clf_mlp = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', MLPClassifier(max_iter=1000)),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_mlp.fit(X_train, y_train)
    y_hat = clf_mlp.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Multilayer Perceptron Neural Network Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Multilayer Perceptron Neural Network Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9811628995485497 and its std: 0.002952109087357548
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.9255790975395456 and its std: 0.012047429218171818


In [38]:
clf_rf = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', RandomForestClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_rf.fit(X_train, y_train)
    y_hat = clf_rf.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Random Forest Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Random Forest Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9717443493228247 and its std: 0.0063913030441011725
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8821780352132119 and its std: 0.02883904899863457


In [39]:
clf_et = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classify', ExtraTreesClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_et.fit(X_train, y_train)
    y_hat = clf_et.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Extra Trees Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Extra Trees Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9733123461823012 and its std: 0.004278087860989827
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8920377444929821 and its std: 0.01910041898827289


In [40]:
clf_gb = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('scale', RobustScaler(with_centering=False)),
    ('classify', GradientBoostingClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_gb.fit(X_train, y_train)
    y_hat = clf_gb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Gradient Boosting Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Gradient Boosting Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9620993190499917 and its std: 0.0031206154519159697
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8404200870363232 and its std: 0.0158255603951531


In [41]:
clf_xgb = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('scale', RobustScaler(with_centering=False)),
    ('classify', XGBClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_xgb.fit(X_train, y_train)
    y_hat = clf_xgb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "XGBoost Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

XGBoost Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9668075875848672 and its std: 0.004180786756706938
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8650760033684426 and its std: 0.017893574576875144


In [42]:
decision_scores = ['cv_accuracy', 'cv_f1_score']
for score_type in decision_scores:
    models_by_score = {}
    for model_name, metrics in model_metrics['metrics'].items():
        models_by_score[metrics[score_type]] = model_name
    scores = np.array(list(models_by_score.keys()))
    max_score = scores.max()
    best_scoring_model_name = models_by_score[max_score]
    print(f"Model with best {score_type}: {best_scoring_model_name}")
    print(f"\tAverage Accuracy: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy_std']}")
    print(f"\tAverage F1-Score: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score_std']}")


Model with best cv_accuracy: Multilayer Perceptron Neural Network Classifier
	Average Accuracy: 0.9811628995485497 with Standard Deviation: 0.002952109087357548
	Average F1-Score: 0.9255790975395456 with Standard Deviation: 0.012047429218171818
Model with best cv_f1_score: Multilayer Perceptron Neural Network Classifier
	Average Accuracy: 0.9811628995485497 with Standard Deviation: 0.002952109087357548
	Average F1-Score: 0.9255790975395456 with Standard Deviation: 0.012047429218171818


A multilayer perceptron neural network ultimately showed the most consistent best performance when constructing a model based on the word content of the sms messages. It will be chosen for hyperparameter tuning.

In [43]:
classifier = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MLPClassifier(max_iter=1000)),
])

param_grid = {
    'classifier__hidden_layer_sizes': [(100,), (50, 50)],
    'classifier__activation': ['relu'],
    'classifier__solver': ['adam'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__learning_rate': ['constant', 'adaptive']
}

grid = GridSearchCV(classifier, param_grid, cv=n_splits, scoring='f1')
grid.fit(X_train_full, y_train_full)

print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)

Best parameters: {'classifier__activation': 'relu', 'classifier__alpha': 0.001, 'classifier__hidden_layer_sizes': (100,), 'classifier__learning_rate': 'constant', 'classifier__solver': 'adam'}
Best cross-validation score: 0.9278587763005082


In [44]:
classifier = Pipeline([
    ('text_cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MLPClassifier(
        max_iter=1000,
        hidden_layer_sizes=grid.best_params_['classifier__hidden_layer_sizes'],
        activation=grid.best_params_['classifier__activation'],
        solver=grid.best_params_['classifier__solver'],
        alpha=grid.best_params_['classifier__alpha'],
        learning_rate=grid.best_params_['classifier__learning_rate']
    )),
])
classifier.fit(X_train_full, y_train_full)
y_hat = classifier.predict(X_test)
report_results = classification_report(y_test, y_hat, output_dict=True)
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))
print(f"Accuracy: {report_results['accuracy']}")
print(f"F1-Score: {report_results['1']['f1-score']}")

[[959   7]
 [ 16 133]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       966
           1       0.95      0.89      0.92       149

    accuracy                           0.98      1115
   macro avg       0.97      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.979372197309417
F1-Score: 0.9204152249134949


The word-based model appears to perform better than the counts-based model. Let's examine the performance of models combining these two feature sets and see how they perform.

## Model Set 3: Combination of String Feature Counts and Word Counts

In [45]:
class TextCounter():

    def __init__(self):
        self.count_types = [
            CountTypes.CHAR,
            CountTypes.PUNC,
            CountTypes.NUM,
            CountTypes.UPPER
        ]
        self.count_methods = [
            self._get_char_count,
            self._get_punc_count,
            self._get_numeric_count,
            self._get_uppercase_count
        ]

    def fit(self, X, y):
        return self

    def fit_transform(self, X, y):
        return self.count_features(X)

    def transform(self, X):
        return self.count_features(X)

    def count_features(self, X):
        # generates features dict to store values in lists organized by count type
        features = {}
        for count_type in self.count_types:
            features[count_type] = []

        # for each body of text, each count method is performed and values are stored in lists in the features dict organized by count type
        for text in X:
            for count_method in self.count_methods:
                count_type, count_val = count_method(text)
                features[count_type].append(count_val)

        # converts dict into 2d list
        features_as_2d_list = []
        for count_vals in features.values():
            features_as_2d_list.append(count_vals)

        # returns array with data organized by row instead of by column
        return np.array(features_as_2d_list).transpose()

    def _get_char_count(self, s: str) -> int:
        return (CountTypes.CHAR, len(s))

    def _get_punc_count(self, s: str) -> int:
        count = 0
        for char in s:
            count += 1 if char in string.punctuation else 0
        return (CountTypes.PUNC, count)

    def _get_numeric_count(self, s: str) -> int:
        count = 0
        tokens = word_tokenize(s)
        for word in tokens:
            count += 1 if word.isnumeric() else 0
        return (CountTypes.NUM, count)

    def _get_uppercase_count(self, s: str) -> int:
        count = 0
        for char in s:
            count += 1 if char.isupper() else 0
        return (CountTypes.UPPER, count)


class CountTypes:
    CHAR = 'char_count'
    PUNC = 'punc_count'
    NUM = 'numeric_count'
    UPPER = 'upper_count'


In [46]:
class TextCleaner():

    def __init__(self):
        self.punct_table = str.maketrans('', '', string.punctuation)
        self.stopwords = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.transformations = [
            self._strip_punct,
            self._convert_to_lowercase,
            self._remove_stopwords,
            self._remove_numbers,
            self._remove_special_characters,
            self._lemmatize
        ]

    def fit(self, X, y):
        return self

    def fit_transform(self, X, y):
        return self.clean_text(X)

    def transform(self, X):
        return self.clean_text(X)

    def clean_text(self, X):
        cleaned_text = []
        for text in X:
            for transformation in self.transformations:
                text = transformation(text)
            cleaned_text.append(text)
        return cleaned_text

    def _strip_punct(self, text: str) -> str:
        return text.translate(self.punct_table)

    def _convert_to_lowercase(self, text: str) -> str:
        return text.lower()

    def _remove_stopwords(self, text: str) -> str:
        words = word_tokenize(text)
        words = [w for w in words if w not in self.stopwords]
        return ' '.join(words)

    def _remove_numbers(self, text: str) -> str:
        words = word_tokenize(text)
        words = [w for w in words if not re.search(r'\d', w)]
        return ' '.join(words)

    def _remove_special_characters(self, text: str) -> str:
        pattern = r'[^a-zA-Z0-9\s]'
        return re.sub(pattern, '', text)

    def _lemmatize(self, text: str) -> str:
        words = word_tokenize(text)
        words = [self.lemmatizer.lemmatize(w) for w in words]
        return ' '.join(words)


In [47]:
messages_df

,Labels,Messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,1,This is the 2nd time we have tried 2 contact u...
5570,0,Will ü b going to esplanade fr home?
5571,0,"Pity, * was in mood for that. So...any other s..."
5572,0,The guy did some bitching but I acted like i'd...


In [48]:
feature_counts_pipe = Pipeline([
    ('counter', TextCounter()),
    ('scaler', RobustScaler())
])

vectorizer_pipe = Pipeline([
    ('cleaner', TextCleaner()),
    ('vectorizer', TfidfVectorizer())
])

In [49]:
str_feature_counts = pd.DataFrame(feature_counts_pipe.fit_transform(messages_df['Messages']), columns=[CountTypes.CHAR, CountTypes.PUNC, CountTypes.NUM, CountTypes.UPPER])
str_feature_counts

,char_count,punc_count,numeric_count,upper_count
0,0.569767,1.50,0.0,0.333333
1,-0.383721,0.75,0.0,0.000000
2,1.081395,0.75,3.0,2.666667
3,-0.151163,0.75,0.0,0.000000
4,-0.011628,-0.25,0.0,0.000000
...,...,...,...,...
5569,1.139535,1.25,3.0,2.333333
5570,-0.302326,-0.50,0.0,-0.333333
5571,-0.058140,1.00,0.0,0.000000
5572,0.732558,-0.50,0.0,0.000000


In [50]:
word_counts = pd.DataFrame(vectorizer_pipe.fit_transform(messages_df['Messages']).toarray(), columns=vectorizer_pipe.named_steps['vectorizer'].get_feature_names_out())
word_counts

,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,...,zebra,zed,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
X = str_feature_counts.join(word_counts)
y = messages_df['Labels']

In [52]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=256)
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

In [53]:
clf_lr = Pipeline([
    ('normalize', Normalizer()),
    ('classify', LogisticRegression()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_lr.fit(X_train, y_train)
    y_hat = clf_lr.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Logistic Regression Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")


Logistic Regression Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9567186564197028 and its std: 0.005946400640017153
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8300742304297934 and its std: 0.024064596528025995


In [54]:
clf_dt = Pipeline([
    ('classify', DecisionTreeClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_dt.fit(X_train, y_train)
    y_hat = clf_dt.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Decision Tree Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Decision Tree Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9683818755567636 and its std: 0.010827017192441054
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8797614070890314 and its std: 0.04157707276971253


In [55]:
clf_nb = Pipeline([
    ('classify', GaussianNB())
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_nb.fit(X_train, y_train)
    y_hat = clf_nb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Gaussian Naive Bayes Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Gaussian Naive Bayes Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.8878674638764323 and its std: 0.007923838736013692
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.6700361737936401 and its std: 0.016733896082594846


In [56]:
k_neighbor_min = 1
k_neighbor_max = 20

knn_classifier_results = {}
max_accuracy = 0
max_f1_score = 0 
for k_neighbors in range(k_neighbor_min,k_neighbor_max+1):
    clf_knn = Pipeline([
        ('classify', KNeighborsClassifier(n_neighbors=k_neighbors)),
    ])
    cv_accuracies = np.array([])
    cv_spam_f1_scores = np.array([])
    for train_index, val_index in skf.split(X_train_full, y_train_full):
        X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
        y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

        clf_knn.fit(X_train, y_train)
        y_hat = clf_nb.predict(X_val)
        report_results = classification_report(y_val, y_hat, output_dict=True)
        cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
        cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

    mean_accuracy = cv_accuracies.mean()
    std_accuracy = cv_accuracies.std()
    mean_f1_score = cv_spam_f1_scores.mean()
    std_f1_score = cv_spam_f1_scores.std()

    if max_accuracy <= mean_accuracy:
        max_accuracy = mean_accuracy
    if max_f1_score <= mean_f1_score:
        max_f1_score = mean_f1_score

    knn_classifier_results[k_neighbors] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

for k_neighbor_val, cv_results in knn_classifier_results.items():
    if cv_results['cv_f1_score'] >= max_f1_score:
        model_name = f"K-Nearest Neighbors Classifier (k={k_neighbor_val})"
        model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

        print(model_name)
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {cv_results['cv_accuracy']} and its std: {cv_results['cv_accuracy_std']}")
        print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {cv_results['cv_f1_score']} and its std: {cv_results['cv_f1_score_std']}")


K-Nearest Neighbors Classifier (k=17)
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9396697417624175 and its std: 0.0089747509318852
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8121912003883238 and its std: 0.022407584081356617


In [57]:
clf_svc = Pipeline([
    ('classify', SVC()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_svc.fit(X_train, y_train)
    y_hat = clf_svc.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Support Vector Machine Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Support Vector Machine Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.960530063968031 and its std: 0.005728933805557085
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.837918426180339 and its std: 0.02331637087556264


In [58]:
clf_mlp = Pipeline([
    ('classify', MLPClassifier(max_iter=1000)),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_mlp.fit(X_train, y_train)
    y_hat = clf_mlp.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Multilayer Perceptron Neural Network Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Multilayer Perceptron Neural Network Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9876663999235001 and its std: 0.004071133927362725
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.9524897833935932 and its std: 0.016052730835191806


In [59]:
clf_rf = Pipeline([
    ('classify', RandomForestClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_rf.fit(X_train, y_train)
    y_hat = clf_rf.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Random Forest Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Random Forest Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9789189855707046 and its std: 0.00681571742982477
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.9139471134361077 and its std: 0.030059649983912855


In [60]:
clf_et = Pipeline([
    ('classify', ExtraTreesClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_et.fit(X_train, y_train)
    y_hat = clf_et.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Extra Trees Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Extra Trees Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9780228795176479 and its std: 0.005139152486103386
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.9102643024148309 and its std: 0.022691036130191477


In [61]:
clf_gb = Pipeline([
    ('classify', GradientBoostingClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_gb.fit(X_train, y_train)
    y_hat = clf_gb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "Gradient Boosting Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

Gradient Boosting Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9726384422199071 and its std: 0.00405762345823216
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.8921592127929671 and its std: 0.016251636084839222


In [62]:
clf_xgb = Pipeline([
    ('classify', XGBClassifier()),
])
cv_accuracies = np.array([])
cv_spam_f1_scores = np.array([])
for train_index, val_index in skf.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    clf_xgb.fit(X_train, y_train)
    y_hat = clf_xgb.predict(X_val)
    report_results = classification_report(y_val, y_hat, output_dict=True)
    cv_accuracies = np.append(cv_accuracies, report_results['accuracy'])
    cv_spam_f1_scores = np.append(cv_spam_f1_scores, report_results['1']['f1-score'])

mean_accuracy = cv_accuracies.mean()
std_accuracy = cv_accuracies.std()
mean_f1_score = cv_spam_f1_scores.mean()
std_f1_score = cv_spam_f1_scores.std()

model_name = "XGBoost Classifier"
model_metrics['metrics'][model_name] = {'cv_accuracy': mean_accuracy, 'cv_accuracy_std': std_accuracy, 'cv_f1_score': mean_f1_score, 'cv_f1_score_std': std_f1_score}

print(model_name)
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Accuracy: {mean_accuracy} and its std: {std_accuracy}")
print(f"Stratified K-Fold ({n_splits}) Cross Validation Average Spam F1-Score: {mean_f1_score} and its std: {std_f1_score}")

XGBoost Classifier
Stratified K-Fold (5) Cross Validation Average Accuracy: 0.9775736940908839 and its std: 0.0067256672755889564
Stratified K-Fold (5) Cross Validation Average Spam F1-Score: 0.9126863716516876 and its std: 0.027409942803528563


In [63]:
decision_scores = ['cv_accuracy', 'cv_f1_score']
for score_type in decision_scores:
    models_by_score = {}
    for model_name, metrics in model_metrics['metrics'].items():
        models_by_score[metrics[score_type]] = model_name
    scores = np.array(list(models_by_score.keys()))
    max_score = scores.max()
    best_scoring_model_name = models_by_score[max_score]
    print(f"Model with best {score_type}: {best_scoring_model_name}")
    print(f"\tAverage Accuracy: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_accuracy_std']}")
    print(f"\tAverage F1-Score: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score']} with Standard Deviation: {model_metrics['metrics'][best_scoring_model_name]['cv_f1_score_std']}")


Model with best cv_accuracy: Multilayer Perceptron Neural Network Classifier
	Average Accuracy: 0.9876663999235001 with Standard Deviation: 0.004071133927362725
	Average F1-Score: 0.9524897833935932 with Standard Deviation: 0.016052730835191806
Model with best cv_f1_score: Multilayer Perceptron Neural Network Classifier
	Average Accuracy: 0.9876663999235001 with Standard Deviation: 0.004071133927362725
	Average F1-Score: 0.9524897833935932 with Standard Deviation: 0.016052730835191806


In [64]:
classifier = Pipeline([
    ('classifier', MLPClassifier(max_iter=1000)),
])

param_grid = {
    'classifier__hidden_layer_sizes': [(100,), (50, 50)],
    'classifier__activation': ['relu'],
    'classifier__solver': ['adam'],
    'classifier__alpha': [0.0001, 0.001],
    'classifier__learning_rate': ['constant', 'adaptive']
}

grid = GridSearchCV(classifier, param_grid, cv=n_splits, scoring='f1')
grid.fit(X_train_full, y_train_full)

print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)

In [ ]:
classifier = Pipeline([
    ('classifier', MLPClassifier(
        max_iter=1000,
        hidden_layer_sizes=grid.best_params_['classifier__hidden_layer_sizes'],
        activation=grid.best_params_['classifier__activation'],
        solver=grid.best_params_['classifier__solver'],
        alpha=grid.best_params_['classifier__alpha'],
        learning_rate=grid.best_params_['classifier__learning_rate']
    )),
])
classifier.fit(X_train_full, y_train_full)
y_hat = classifier.predict(X_test)
report_results = classification_report(y_test, y_hat, output_dict=True)
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))
print(f"Accuracy: {report_results['accuracy']}")
print(f"F1-Score: {report_results['1']['f1-score']}")

[[958   8]
 [ 10 139]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.95      0.93      0.94       149

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Accuracy: 0.9838565022421525
F1-Score: 0.9391891891891893


## Results

Between the counts-based model and the word-based model, the word-based model shows a better f1-score and accuracy. The combined model shows performance that is not different enough from the word-based model to warrant using.